In [1]:
import pandas as pd
import datetime
import numpy as np
import lux

In [2]:
apart = pd.read_csv('apart(2016~2020).csv') # 2016-2020 아파트 데이터 불러오기

C:\Users\meteo\.conda\envs\final\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (1,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
apart.drop(['실거래가아이디','지번코드','시군구코드','법정동코드','업무구분코드','업무구분','물건번호','대지권면적','관리구분코드','건물주용도코드','건물주용도'], axis=1, inplace=True) #미사용 열 삭제

In [4]:
apart_2021 = pd.read_csv('apart(2021).csv') # 2021 아파트 데이터 불러오기

In [5]:
apart_2021 = apart_2021[apart_2021['해제사유발생일'].isnull()] # 계약 해제사유발생 건 삭제
apart_2021.drop(['번지','본번','부번','계약일','도로명','해제사유발생일','거래유형','중개사소재지'],axis=1, inplace=True) # 불필요 열 삭제

apart_2021['자치구명'] = apart_2021['시군구'].str.split(" ").str[1] # 자치구명 추출
apart_2021['법정동명'] = apart_2021['시군구'].str.split(" ").str[2] # 법정동명 추출
apart_2021.drop(['시군구'], axis='columns', inplace=True) # 시군구 열 삭제

apart_2021['계약년월'] = apart_2021['계약년월'] = 2021 # 계약년월 전체 2021년으로 대체

apart_2021['거래금액(만원)'] = apart_2021['거래금액(만원)'].str.replace(",", '') # 거래금액 열 내 ',' 제거
apart_2021['거래금액(만원)'] = apart_2021['거래금액(만원)'].astype(int) # 거래금액 열 타입 변경(object > int)
apart_2021['거래금액(만원)'] = apart_2021['거래금액(만원)'].apply(lambda x : x * 10000) # 거래금액 열 단위(만원 > 원) 변경

apart_2021 = apart_2021.rename(columns={'단지명':'건물명', '전용면적(㎡)':'건물면적', '거래금액(만원)':'물건금액', '층':'층정보', '계약년월':'신고년도'}) # 열 이름 변경
apart_2021 = apart_2021[['자치구명','법정동명','신고년도','건물면적','층정보','물건금액','건축년도','건물명']] # 열 순서 변경

In [6]:
apart_all = pd.concat([apart, apart_2021]) # 2016-2020 아파트 데이터 + 2021 아파트 데이터 Concat
apart_all.reset_index(inplace=True,drop=True) # Concat 이후 인덱스 초기화

In [7]:
floor_drop_index = apart_all[apart_all['층정보'] == 0].index # 층정보 결측치 삭제
apart_all.drop(floor_drop_index, inplace=True)
const_year_index = apart_all[apart_all['건축년도'] == 0].index # 건축년도 결측치 삭제
apart_all.drop(const_year_index, inplace=True)

apart_all['경과년도'] = apart_all['건축년도'].apply(lambda x : int(datetime.datetime.now().year) - x) # 현재년도 - 건축년도
apart_all['경과년도'] = apart_all[['경과년도']].replace(int(datetime.datetime.now().year),0) # 0 값으로 생긴 현재년도 값 0으로 대체

brands = ['래미안','힐스테이트','자이','더샵','#','푸르지오','힐스테이트','롯데캐슬','아크로','아이파크','IPARK','에스케이뷰','SKVIEW'] # 시공사 브랜드 Top 10
brand = '|'.join(brands) # contains 기능을 활용해 한번에 찾아낼 수 있도록 List > Str 형태로 변형하며 브랜드 사이에 |(or) 입력
apart_all['브랜드점수'] = apart_all['건물명'].str.contains(brand) # 건물명에 Top10 브랜드가 있는 경우 True 아니면 False
apart_all['브랜드점수'] = apart_all['브랜드점수'].apply(lambda x : 1 if x == True else 0)

apart_all['법정동명'] = apart_all['법정동명'].apply(lambda x : x[:-2] if x[-1] in '가' else x) # 법정동명 값 중 OO동 뒤에 'X가' 같은 구획이 붙은 경우 삭제

래미안|힐스테이트|자이|더샵|#|푸르지오|힐스테이트|롯데캐슬|아크로|아이파크|IPARK|에스케이뷰|SKVIEW


In [17]:
apart_priceindex = pd.read_csv('매매가격지수(아파트)_통합.csv', index_col='지역') # 매매가격지수 데이터 불러오기

# 기존 아파트 실거래 데이터에서 신고년도/구 가 일치하는 매매가격지수 데이터를 새로운 열로 추가

price_index_list = []

for idx, row in apart_all.iterrows():
        price_index_list.append(apart_priceindex[str(row['신고년도'])][row['자치구명']])
        
apart_all['매매지수'] = price_index_list

# for idx, row in apart_all.iterrows():
#     if idx < 10:
#         print(row['자치구명'])
#         print(row['신고년도'])
#         print(apart_priceindex[str(row['신고년도'])][row['자치구명']])
#         print()
#     else:
#         break

In [18]:
apart_all.info()

<class 'lux.core.frame.LuxDataFrame'>
Int64Index: 493729 entries, 0 to 518344
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   자치구명    493729 non-null  object 
 1   법정동명    493729 non-null  object 
 2   신고년도    493729 non-null  int64  
 3   건물면적    493729 non-null  float64
 4   층정보     493729 non-null  int64  
 5   물건금액    493729 non-null  int64  
 6   건축년도    493729 non-null  int64  
 7   건물명     493729 non-null  object 
 8   경과년도    493729 non-null  int64  
 9   브랜드점수   493729 non-null  int64  
 10  매매지수    493729 non-null  float64
dtypes: float64(2), int64(6), object(3)
memory usage: 45.2+ MB


In [11]:
# lux 데이터 시각화 1
apart_all.intent = ['물건금액'] # 물건금액과 관련된 시각화 자료 확인
apart_all

C:\Users\meteo\.conda\envs\final\lib\site-packages\lux\executor\PandasExecutor.py:422: UserWarning:
Lux detects that attributes ['신고년도', '건축년도'] may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Datetime objects.
For example, you can convert a Year attribute (e.g., 1998, 1971, 1982) using pd.to_datetime by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['신고년도'] = pd.to_datetime(df['신고년도'], format='<replace-with-datetime-format>')
	df['건축년도'] = pd.to_datetime(df['건축년도'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
If 건축년도 is not a temporal attribute, please use override Lux's automatically detected type:
	df.set_data_type({'건축년도':'quantitative'})


Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [ ]:
apart_all.to_csv('apart_all(2016-2021).csv', encoding='utf-8', index=False) # 전처리 완료 데이터 csv 저장